In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from nltk.corpus import stopwords
import re
import string

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
sample=pd.read_csv('Sample Submission.csv')

In [3]:
train.head()

,Text_ID,Product_Description,Product_Type,Sentiment
0,3057,The Web DesignerÛªs Guide to iOS (and Android...,9,2
1,6254,RT @mention Line for iPad 2 is longer today th...,9,2
2,8212,Crazy that Apple is opening a temporary store ...,9,2
3,4422,The lesson from Google One Pass: In this digit...,9,2
4,5526,RT @mention At the panel: &quot;Your mom has a...,9,2


In [4]:
test.head()

,Text_ID,Product_Description,Product_Type
0,5786,RT @mention Going to #SXSW? The new iPhone gui...,7
1,5363,RT @mention 95% of iPhone and Droid apps have ...,9
2,6716,RT @mention Thank you to @mention for letting ...,9
3,4339,#Thanks @mention we're lovin' the @mention app...,7
4,66,At #sxsw? @mention / @mention wanna buy you a ...,9


In [5]:
sample.head()

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [7]:
px.bar(x=train['Product_Type'].value_counts().index,y=train['Product_Type'].value_counts())

In [8]:
px.bar(x=train['Sentiment'].value_counts().index,y=train['Sentiment'].value_counts())

In [5]:
def clean(text):
    stop=set(stopwords.words('english'))
    s=[]
    for w in text.split():
        if w.lower() not in stop:
            s.append(w)
    s=' '.join([word for word in s])
    url=re.compile(r'https?://\S+|www\.\S+')
    k=url.sub(r'',s)
    html=re.compile(r'<.*?>')
    k=html.sub(r'',k)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    k=emoji_pattern.sub(r'',k)
    table=str.maketrans('','',string.punctuation)
    k=k.translate(table)
    return k.lower()

In [6]:
train['Product_Description']=train['Product_Description'].apply(clean)
test['Product_Description']=test['Product_Description'].apply(clean)

In [7]:
train.drop(['Text_ID'],axis=1,inplace=True)
test.drop(['Text_ID'],axis=1,inplace=True)

In [8]:
train

,Product_Description,Product_Type,Sentiment
0,web designerûªs guide ios and android apps to...,9,2
1,rt mention line ipad 2 longer today yesterday ...,9,2
2,crazy apple opening temporary store austin tom...,9,2
3,lesson google one pass digital environment use...,9,2
4,rt mention panel quotyour mom ipad designing b...,9,2
...,...,...,...
6359,thanks mention hosting event sxsw sxswi iwanta...,9,2
6360,sxsw iphone app awesome ipad app crashes every...,7,1
6361,rt mention rt mention download 20 free tracks ...,9,2
6362,oh texas social media awards you need ask moth...,9,2


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [10]:
train

,Product_Description,Product_Type,Sentiment
0,web designerûªs guide ios android apps today ...,9,2
1,rt mention line ipad 2 longer today yesterday ...,9,2
2,crazy apple opening temporary store austin tom...,9,2
3,lesson google one pass digital environment use...,9,2
4,rt mention panel quotyour mom ipad designing b...,9,2
...,...,...,...
6359,thanks mention hosting event sxsw sxswi iwanta...,9,2
6360,sxsw iphone app awesome ipad app crashes every...,7,1
6361,rt mention rt mention download 20 free tracks ...,9,2
6362,oh texas social media awards need ask mother a...,9,2


In [11]:
X=train.drop(['Sentiment','Product_Type'],axis=1)
y=train['Sentiment']

In [12]:
X

,Product_Description
0,web designerûªs guide ios android apps today ...
1,rt mention line ipad 2 longer today yesterday ...
2,crazy apple opening temporary store austin tom...
3,lesson google one pass digital environment use...
4,rt mention panel quotyour mom ipad designing b...
...,...
6359,thanks mention hosting event sxsw sxswi iwanta...
6360,sxsw iphone app awesome ipad app crashes every...
6361,rt mention rt mention download 20 free tracks ...
6362,oh texas social media awards need ask mother a...


In [10]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier
from sklearn import metrics
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [152]:
fold=StratifiedKFold(n_splits=5)

In [29]:
preprocess = ColumnTransformer(
    [('description_tfidf', TfidfVectorizer(ngram_range=(1,3)), 'Product_Description')],
    remainder='passthrough')

In [30]:
X=train[['Product_Description','Product_Type']]
y=train['Sentiment']
model = CatBoostClassifier(loss_function='Logloss')

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
cb = CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="neg_log_loss", cv = 3)
pipe=make_pipeline(preprocess,cb_model)
pipe.fit(X,y)




 37s	remaining: 36m 59s
93:	learn: 0.4517971	total: 16m 48s	remaining: 36m 50s
94:	learn: 0.4510412	total: 16m 59s	remaining: 36m 40s
95:	learn: 0.4505870	total: 17m 11s	remaining: 36m 31s
96:	learn: 0.4499382	total: 17m 22s	remaining: 36m 21s
97:	learn: 0.4495749	total: 17m 33s	remaining: 36m 11s
98:	learn: 0.4488764	total: 17m 44s	remaining: 36m 2s
99:	learn: 0.4484308	total: 17m 56s	remaining: 35m 52s
100:	learn: 0.4480092	total: 18m 7s	remaining: 35m 41s
101:	learn: 0.4476013	total: 18m 17s	remaining: 35m 29s
102:	learn: 0.4471884	total: 18m 28s	remaining: 35m 19s
103:	learn: 0.4468858	total: 18m 38s	remaining: 35m 7s
104:	learn: 0.4461816	total: 18m 49s	remaining: 34m 58s
105:	learn: 0.4457643	total: 19m 3s	remaining: 34m 53s
106:	learn: 0.4449725	total: 19m 19s	remaining: 34m 50s
107:	learn: 0.4446856	total: 19m 31s	remaining: 34m 43s
108:	learn: 0.4443240	total: 19m 43s	remaining: 34m 34s
109:	learn: 0.4440363	total: 19m 56s	remaining: 34m 26s
110:	learn: 0.4437427	total: 20m 11

KeyboardInterrupt: 

In [15]:
grid.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [31]:
pred=pipe.predict_proba(test[['Product_Description','Product_Type']])

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [25]:
pred

array([[0.00454784, 0.13137745, 0.0300057 , 0.834069  ],
       [0.02603361, 0.02478693, 0.9146843 , 0.03449517],
       [0.02072421, 0.00904525, 0.91853106, 0.05169948],
       ...,
       [0.00536756, 0.12601714, 0.03633871, 0.83227659],
       [0.02311405, 0.01172248, 0.91010323, 0.05506025],
       [0.0087685 , 0.17319578, 0.02831869, 0.78971702]])

In [26]:
cols=sample.columns
sample[cols]=pred

In [27]:
sample

,0,1,2,3
0,0.004548,0.131377,0.030006,0.834069
1,0.026034,0.024787,0.914684,0.034495
2,0.020724,0.009045,0.918531,0.051699
3,0.005190,0.113118,0.041307,0.840384
4,0.016095,0.007589,0.928392,0.047924
...,...,...,...,...
2723,0.023333,0.011318,0.907677,0.057672
2724,0.017272,0.006909,0.924333,0.051486
2725,0.005368,0.126017,0.036339,0.832277
2726,0.023114,0.011722,0.910103,0.055060


In [28]:
sample.to_csv('pred11.csv',index=False)